# Aula 02

## Erro mais comum ao iniciar em ML

Imagine que vamos preencher oa idade média dos clientes:
 - **Erro:** Calcular a média de **todos** oe dados (treino + teste) e preencher.
 - **Por que está errado?** Você utilizou informações do teste (que não devem ser conhecidos pelo modelo) para preparar o treino. Pense que é como você estivesse colando na prova.
 - **A regra de ouro:** Qualquer cálculo de média, desvio padrão ou escalar deve ser aprendido (`.fit()`) **apenas nos dados de treino** e depois aplicado (`.transform()`) nos dados de teste.

## Criando o dataset "sujo"

In [8]:
# Criar um dataset com dados faltantes
import numpy as np
import pandas as pd

dados = {
    'idade': [25, 30, np.nan, 40, 22, 35, np.nan, 45, 28, 50],
    'salario': [5000, 6000, 5500, np.nan, 4500, 7000, 6200, 8000, 5800, np.nan],
    'cidade': ['SP', 'RJ', 'SP', 'BH', 'RJ', 'SP', 'BH', 'SP', 'RJ', 'SP'],
    'genero': ['M', 'F', 'F', 'M', 'M', 'F', 'F', 'M', 'M', 'F'],
    'comprou': ['Não', 'Sim', 'Não', 'Sim', 'Não',
               'Sim', 'Não', 'Sim', 'Não', 'Sim']
}

# Criar o DataFrame
df = pd.DataFrame(dados)

# Separar as perguntas da resposta
X = df.drop('comprou', axis=1)
y = df['comprou']

In [9]:
# Dividir antes de arrumar os dados
from sklearn.model_selection import train_test_split

X_treino, X_teste, y_treino, y_teste = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

X_treino

,idade,salario,cidade,genero
5,35.0,7000.0,SP,F
0,25.0,5000.0,SP,M
7,45.0,8000.0,SP,M
2,NaN,5500.0,SP,F
9,50.0,NaN,SP,F
4,22.0,4500.0,RJ,M
3,40.0,NaN,BH,M
6,NaN,6200.0,BH,F


## Lidando com dados faltantes

Modelos matemáticos não aceitam o valor vazio (`NaN`). Precisamos preencher esse espaços com dados úteis. O nome técnico dessa etapa é **imputação**.

### SimplerImputer (o básico)

In [10]:
from sklearn.impute import SimpleImputer

# Definir a estratégia (mean, median, most_frequent, constant)
imputer_idade = SimpleImputer(strategy='mean')

# Aplicar a regra de ouro:
# .fit() no TREINO (aprende a média do treino)
# .transform() no TREINO e TESTE (aplica a média aprendida)
X_treino['idade'] = imputer_idade.fit_transform(X_treino[['idade']])
X_teste['idade'] = imputer_idade.fit_transform(X_teste[['idade']])

print('Idade tratada no treino')
X_treino['idade'].values

Idade tratada no treino


array([35.        , 25.        , 45.        , 36.16666667, 50.        ,
       22.        , 40.        , 36.16666667])

### KNNImputer (avançado)

E se calcular a média for muito "burro"? O **KNNImputer** olha para as linhas mais parecidas. Se um cliente tem perfil parecido com o João, ele provavelmente tem o salário parecido com o do João.

In [11]:
from sklearn.impute import KNNImputer

# Utilizar 3 vizinhos para estimar o salário
imputer_knn = KNNImputer(n_neighbors=3)

# Regra de ouro
X_treino['salario'] = imputer_knn.fit_transform(X_treino[['salario']])
X_teste['salario'] = imputer_knn.transform(X_teste[['salario']])

print('Salário tratato no treino')
X_treino['salario'].values

Salário tratato no treino


array([7000.        , 5000.        , 8000.        , 5500.        ,
       6033.33333333, 4500.        , 6033.33333333, 6200.        ])

## Codificação de variáveis categóricas (Encoding)

Os modelos não sabem "ler" palavras, somente números; uma vez que são usados cálculos matemáticos para chegar ao resultado desejado, logo, usar palavras não faz sentido. Por isso, temos que transformar as palavras (texto) em números.

### OrdinalEncoder vs LabelEncoder

Muito cuidado com a confusão:
- **LabelEncoder:** Use **APENAS** para o a resposta (alvo) `y`. Nunca use nas perguntas (features) `X`.
- **OrdinalEncoder:** Use para as perguntas (features) `X` quando existe uma ordem lógica, por exemplo: ruim, médio, bom -> 0, 1, 2.

In [12]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_treino = le.fit_transform(y_treino)
y_teste = le.transform(y_teste)

print(f'Resposta transformada: {y_treino}')  # 0=Não, 1=Sim

Resposta transformada: [1 0 1 0 1 0 1 0]


###OneHotEncoder

Para a coluna `'cidade'` e `'genero'`, não existe ordem (SP não é maior que RJ). Se usarmos 1, 2, 3, o modelo acahrá que 3 vale mais do que 1. Para isso usamos o **OneHotEncoder**. Ele cria uma coluna para cada opção com 0s e 1s.

In [13]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    sparse_output=False,  # ver o array numpy visualmente
    handle_unknown='ignore'  # se esxistir uma categorias desconhecida retorna 0
)

# Regra de ouro
X_treino_cat = ohe.fit_transform(X_treino[['cidade', 'genero']])
X_teste_cat = ohe.transform(X_teste[['cidade', 'genero']])

# Converter em um DF para melhor visualização
colunas_novas = ohe.get_feature_names_out(['cidade', 'genero'])
df_cat_treino = pd.DataFrame(X_treino_cat, columns=colunas_novas, index=X_treino.index)
df_cat_teste = pd.DataFrame(X_teste, columns=colunas_novas, index=X_teste.index)

df_cat_treino

,cidade_BH,cidade_RJ,cidade_SP,genero_F,genero_M
5,0.0,0.0,1.0,1.0,0.0
0,0.0,0.0,1.0,0.0,1.0
7,0.0,0.0,1.0,0.0,1.0
2,0.0,0.0,1.0,1.0,0.0
9,0.0,0.0,1.0,1.0,0.0
4,0.0,1.0,0.0,0.0,1.0
3,1.0,0.0,0.0,0.0,1.0
6,1.0,0.0,0.0,1.0,0.0


In [14]:
# colocar os dados codificados nos dados de treino e teste
X_treino = pd.concat([X_treino.drop(columns=['cidade', 'genero']),
                      df_cat_treino], axis=1)
X_teste = pd.concat([X_teste.drop(columns=['cidade', 'genero']),
                      df_cat_teste], axis=1)

X_treino.head()

,idade,salario,cidade_BH,cidade_RJ,cidade_SP,genero_F,genero_M
5,35.000000,7000.000000,0.0,0.0,1.0,1.0,0.0
0,25.000000,5000.000000,0.0,0.0,1.0,0.0,1.0
7,45.000000,8000.000000,0.0,0.0,1.0,0.0,1.0
2,36.166667,5500.000000,0.0,0.0,1.0,1.0,0.0
9,50.000000,6033.333333,0.0,0.0,1.0,1.0,0.0


## Featuring scaling (colocando na mesma régua)

Olhe para nossos dados: a idade vai de 25 a 50 e o salário vai de 4500 a 8000. Para o algoritmo, a diferença de 1000 no slário é gigante quando comparada a diferença de 10 na idade. O salário vai dominar o modelo. Precisamos colocar tudo na mesma escala.

- **StandardScaler (padronização):** Transforma os dados para que a média seja 0 e o desvio padrão seja 1. É o **padrão da indústria** para a maioria dos algorimos (regressão linear, SVM, redeis neuais).
- **MinMaxScaler:** Esmaga os dados entre 0 e 1. Muito utilizados em processamento de imagem.
- **RobustScaler:** Se alguém ganhar 1 mihão enquanto a média é 5000, o `StandardScaler` falhará. O `RobustScaler` usa a mediana e qurtis, ignorando esse outliers extremos.

### StandartScaler (padronização)

In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

# Escalar somente as colunas numéricas originais
cols_num = ['idade', 'salario']

X_treino[cols_num] = scaler.fit_transform(X_treino[cols_num])
X_teste[cols_num] = scaler.transform(X_teste[cols_num])

X_treino

,idade,salario,cidade_BH,cidade_RJ,cidade_SP,genero_F,genero_M
5,-0.133515,0.935971,0.0,0.0,1.0,1.0,0.0
0,-1.277930,-1.000521,0.0,0.0,1.0,0.0,1.0
7,1.010900,1.904217,0.0,0.0,1.0,0.0,1.0
2,0.000000,-0.516398,0.0,0.0,1.0,1.0,0.0
9,1.583108,0.000000,0.0,0.0,1.0,1.0,0.0
4,-1.621255,-1.484644,0.0,1.0,0.0,0.0,1.0
3,0.438693,0.000000,1.0,0.0,0.0,0.0,1.0
6,0.000000,0.161374,1.0,0.0,0.0,1.0,0.0
